# Manufactured solutions for Elasticity

This is a notebook to play with manufactured solutions for Elasticity.

In [1]:
from __future__ import division
from sympy import *
x, y, z, t = symbols('x y z t')
f, g, h = symbols('f g h', cls=Function)
init_printing()

In [2]:
L = symbols('L')
a1, a2, a3, b1, b2, b3, c1, c2, c3 = symbols('a1 a2 a3 b1 b2 b3 c1 c2 c3')
u0, ux, uy, uz, v0, vx, vy, vz, w0, wx, wy, wz = symbols('u0 u_x u_y u_z\
                                                         v0 v_x v_y v_z\
                                                         w0 w_x w_y w_z')
lamda, mu, rho = symbols('lamda mu rho')

In [3]:
u = u0 + ux*sin(a1*pi*x/L) + uy*sin(a2*pi*y/L) + uz*sin(a3*pi*z/L)
v = v0 + vx*sin(b1*pi*x/L) + vy*sin(b2*pi*y/L) + vz*sin(b3*pi*z/L)
w = w0 + wx*sin(c1*pi*x/L) + wy*sin(c2*pi*y/L) + wz*sin(c3*pi*z/L)

In [4]:
def laplacian(U, X=[x, y, z]):
    lap_U = Matrix([sum([diff(U[k], X[j], 2) for j in range(3)]) for k in range(3)])
    return lap_U  


def div(U, X=[x, y, z]):
    return sum([diff(U[k], X[k]) for k in range(3)])


def grad(f, X=[x, y, z]):
    return Matrix([diff(f, X[k]) for k in range(3)])


def rot(U, X=[x, y, z]):
    return Matrix([diff(U[2], X[1]) - diff(U[1], X[2]),
                   diff(U[0], X[2]) - diff(U[2], X[0]),
                   diff(U[1], X[0]) - diff(U[0], X[1])])

def navier(U, X=[x,y,z], lamda=lamda, mu=mu):
    return mu*laplacian(U, X) + (lamda + mu)*grad(div(U, X), X)


def dt(U, t=t):
    return Matrix([diff(U[k], t) for k in range(3)])

In [5]:
U = [u,v,w]
F = rho*dt(U) - navier(U)

simplify(F)

⎡ 2 ⎛  2               ⎛π⋅a₁⋅x⎞     ⎛  2       ⎛π⋅a₁⋅x⎞     2        ⎛π⋅a₂⋅y⎞ 
⎢π ⋅⎜a₁ ⋅uₓ⋅(λ + μ)⋅sin⎜──────⎟ + μ⋅⎜a₁ ⋅uₓ⋅sin⎜──────⎟ + a₂ ⋅u_y⋅sin⎜──────⎟ 
⎢   ⎝                  ⎝  L   ⎠     ⎝          ⎝  L   ⎠              ⎝  L   ⎠ 
⎢─────────────────────────────────────────────────────────────────────────────
⎢                                                  2                          
⎢                                                 L                           
⎢                                                                             
⎢ 2 ⎛  2                ⎛π⋅b₂⋅y⎞     ⎛  2       ⎛π⋅b₁⋅x⎞     2        ⎛π⋅b₂⋅y⎞
⎢π ⋅⎜b₂ ⋅v_y⋅(λ + μ)⋅sin⎜──────⎟ + μ⋅⎜b₁ ⋅vₓ⋅sin⎜──────⎟ + b₂ ⋅v_y⋅sin⎜──────⎟
⎢   ⎝                   ⎝  L   ⎠     ⎝          ⎝  L   ⎠              ⎝  L   ⎠
⎢─────────────────────────────────────────────────────────────────────────────
⎢                                                   2                         
⎢                                                  L

## Cosserat solid


Let's start with the 2D case

In [6]:
L = symbols('L')
h0, hx, hy = symbols('h0 hx hy')
lamda, mu, rho = symbols('lamda mu rho')
alpha, mu_c, xi, gamma, J = symbols('alpha mu_c xi gamma J')

In [7]:
u = u0 + ux*sin(a1*pi*x/L) + uy*sin(a2*pi*y/L)
v = v0 + vx*sin(b1*pi*x/L) + vy*sin(b2*pi*y/L)
w = 0
h1 = 0
h2 = 0
h3 = h0 + hx*sin(c1*pi*x/L) + hy*sin(c2*pi*y/L)

In [8]:
def coss(U, H, X=[x,y,z], lamda=lamda, mu=mu, mu_c=mu_c, xi=xi, gamma=gamma):
    f_u = (lamda + 2*mu)*grad(div(U)) - (mu + mu_c)*rot(rot(U)) + 2*mu_c*rot(H)
    f_h = (alpha + 2*gamma + 2*xi)*grad(div(H)) - 2*xi*rot(rot(H)) + 2*mu_c*rot(U) - 4*mu_c*Matrix(H)
    return f_u, f_h

In [9]:
U = [u, v, w]
H = [h1, h2, h3]
f_u, f_h = coss(U, H)

In [12]:
simplify(f_u)

⎡   ⎛                 ⎛π⋅c₂⋅y⎞     ⎛  2                 ⎛π⋅a₁⋅x⎞     2        
⎢ π⋅⎜2⋅L⋅c₂⋅hy⋅μ_c⋅cos⎜──────⎟ - π⋅⎜a₁ ⋅uₓ⋅(λ + 2⋅μ)⋅sin⎜──────⎟ + a₂ ⋅u_y⋅(μ 
⎢   ⎝                 ⎝  L   ⎠     ⎝                    ⎝  L   ⎠              
⎢ ────────────────────────────────────────────────────────────────────────────
⎢                                                 2                           
⎢                                                L                            
⎢                                                                             
⎢   ⎛                 ⎛π⋅c₁⋅x⎞     ⎛  2                 ⎛π⋅b₁⋅x⎞     2        
⎢-π⋅⎜2⋅L⋅c₁⋅hx⋅μ_c⋅cos⎜──────⎟ + π⋅⎜b₁ ⋅vₓ⋅(μ + μ_c)⋅sin⎜──────⎟ + b₂ ⋅v_y⋅(λ 
⎢   ⎝                 ⎝  L   ⎠     ⎝                    ⎝  L   ⎠              
⎢─────────────────────────────────────────────────────────────────────────────
⎢                                                 2                           
⎢                                                L  

In [13]:
simplify(f_h)

⎡                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢ ⎛   2     ⎛           ⎛π⋅c₁⋅x⎞         ⎛π⋅c₂⋅y⎞⎞             ⎛          ⎛π⋅a
⎢-⎜4⋅L ⋅μ_c⋅⎜h₀ + hx⋅sin⎜──────⎟ + hy⋅sin⎜──────⎟⎟ + 2⋅π⋅L⋅μ_c⋅⎜a₂⋅u_y⋅cos⎜───
⎢ ⎝         ⎝           ⎝  L   ⎠         ⎝  L   ⎠⎠             ⎝          ⎝  L
⎢─────────────────────────────────────────────────────────────────────────────
⎢                                                                             
⎣                                                                             

0                                                                             
                                                                              
0                                                  

## References

- Malaya, Nicholas, et al. "MASA: a library for verification using manufactured and analytical solutions." Engineering with Computers 29.4 (2013): 487-496.

In [12]:
from IPython.core.display import HTML
def css_styling():
    styles = open('./styles/custom_barba.css', 'r').read()
    return HTML(styles)
css_styling()